In [2]:
import sys
sys.path.insert(0, '/home/toepfer/Documents/Project_PhysNet3/KaiAsparagus')

# Reference Calculation on Local Workstations or Clusters

Asparagus provides a low-level sample calculator ```'shell'``` for local execution on a linux shell (e.g. ```bash```) or ```'slurm'``` for executions with the 'slurm' job manangement system. It is a low-level option, as it requires template files specifically modified for the used quantumchemistry program, calculation options and required system properties. The following tutorial shows two examples with the ```'shell'``` and ```'slurm'``` calculator for a set of ammonia structures stored in an ASE trajectory file or a Asparagus database file.
The first example computes energy, forces and molecular dipole moment of a set of ammonia structures. The difference to the ```Sampler``` example in the sampling tutorial are, more or less, just the calculator arguments ```sample_calculator``` and ```sample_calculator_args```.

## Reference Calculation on Local Workstations with ```'shell'```

The sample calculator ```'shell'``` works as follows. 
The template files in the list ```'files'``` in ```sample_calculator_args``` will be read, modified for the specific sample and written to the respective working directory. The modification are replacing certain tag labels in the template files (keys in ```'files_replace'``` in ```sample_calculator_args``` ) with system specific properties such as the Cartesian positions in a ```xyz``` format (```'$xyz'```). The first file in the template file list (or in ```'execute_file'```) will be executed in the working directory by the command ```bash run_orca.sh```. The shell script file ```run_orca.sh``` will call ```ORCA``` with the system specific input file ```run_orca.inp``` and execute the python script ```run_orca.py``` second that reads the requested system properties from the ```ORCA``` output file and stores them in ```results.json```. 

In [3]:
from asparagus.sample import Sampler
sampler = Sampler(
    config='smpl_nh3.json',
    sample_data_file='smpl_nh3.db',
    sample_systems='ref/meta_nh3.traj',
    sample_calculator='shell',
    sample_calculator_args = {
        'files': [
            'template/shell/run_orca.sh',
            'template/shell/run_orca.inp',
            'template/shell/run_orca.py',
            ],
        'files_replace': {
            '%xyz%': '$xyz',
            '%charge%': '$charge',
            '%multiplicity%': '$multiplicity',
            },
        'result_properties': ['energy', 'forces', 'dipole'],
        'execute_file': 'template/shell/run_orca.sh',
        'command': 'bash',
        'charge': 0,
        'multiplicity': 1,
        'directory': 'shell',
        'result_file': 'results.json',
        'result_file_format': 'json',
        },
    sample_num_threads=4,
    sample_save_trajectory=True,
    )
sampler.run()

INFO:asparagus.settings.config:INFO:
Configuration file path set to 'smpl_nh3.json'!

INFO:asparagus.settings.config:INFO:
Parameter update in 'smpl_nh3.json'
  (overwrite conflicts)
Adding parameter 'sampler_schedule'.
Adding parameter 'sample_counter'.
2 new parameter, 2 added, 0 equal, 0 overwritten

INFO:asparagus.sample.sampler:INFO:
Perform sampling method 'sample' on systems:
   1. 'ref/meta_nh3.traj'

INFO:asparagus.sample.sampler:INFO:
Sampling method 'sample' complete for system from 'ref/meta_nh3.traj!'
2 samples written to 'smpl_nh3.db'.

INFO:asparagus.sample.sampler:INFO:
Sampling method 'sample' complete for system from 'ref/meta_nh3.traj!'
3 samples written to 'smpl_nh3.db'.

INFO:asparagus.sample.sampler:INFO:
Sampling method 'sample' complete for system from 'ref/meta_nh3.traj!'
3 samples written to 'smpl_nh3.db'.

INFO:asparagus.sample.sampler:INFO:
Sampling method 'sample' complete for system from 'ref/meta_nh3.traj!'
3 samples written to 'smpl_nh3.db'.



Most work from the user is probably going into the template files list ```'files'``` in ```sample_calculator_args```. The first file in the list will assumed to be executed by the, e.g., ```bash``` command if not specifically defined by the input argument ```'execute_file'```. In this case for ```ORCA``` it only contains commands to (1) delete eventually old wavefunction files, (2) run ```ORCA``` and (3) run the python file ```run_orca.py```.

run_orca.sh:
```
rm -f run_orca.gbw run_orca.ges
orca run_orca.inp > run_orca.out
python run_orca.py
```

The ```ORCA``` input file ```run_orca.inp``` contains the setup to perform the quantumelectronic computation on PBE level of theory.
Only this file cotains the label tags that will be replaced with system specific input as defined in ```'files_replace'```. Important here is that the dictionary items ```'$xyz'```, ```'$charge'``` and ```'$multiplicity'``` are labels for pre-defined function that take the ASE Atoms object and a info dictionary as input to produce the input line for the element symbols and Cartesian atom positions, the system charge and its electronic spin multiplicity, respectively.

run_orca.inp:
```
! engrad RI PBE D3BJ def2-SVP def2/J TightSCF 
%pal nprocs 1 end 
*xyz %charge% %multiplicity%
%xyz%
*
```
The input file for one sample system will then looks like:
```
! engrad RI PBE D3BJ def2-SVP def2/J TightSCF 
%pal nprocs 1 end 
*xyz 0 1
N     0.07351617  -0.00534278  -0.23603505 
H    -1.03994361  -0.03392835  -0.40634277 
H     0.31140533   0.88603402  -0.62595885 
H     0.48690189  -0.79495547  -0.78437965 
*
```

After ```ORCA``` is run using the modified input file, the output file will be read by the python file ```'run_orca.py'``` which extract the required system properties (see ```'result_properties': ['energy', 'forces', 'dipole']``` in ```sample_calculator_args```) and store the data in a .json file ```'result.json'```. The json file and the stored system properties will be read by Asparagus and handled as any other ASE calculator result.

result.json:
```
{"energy": -1535.4234625489537,
 "dipole": [-0.08876947707208435, 0.02660703014714993, -0.33175706861878895],
 "forces": [
   [-3.235397732194288, -1.474455583716518, 0.3764568818272469],
   [2.949547650701895, -0.22134084646210062, 0.3515452359394994],
   [0.6608213322874061, 0.8358449528315552, -0.7880407905321968],
   [-0.3749712507435909, 0.8599514773470633, 0.06003867276545045]]}
```